In [2]:
#Financial Computing HW3
import numpy as np
from scipy.stats import norm

#input values setting
K = 100
r = 0.1
T = 0.5
num_simulation = 10000
num_repetitions = 20
S0 = [95, 95, 95, 95, 95]
q = [.05, .05, .05, .05, .05]
sigma = [.5, .5, .5, .5, .5]


In [4]:
#Basic Requirement
#Cholesky Decomposition

def cholesky_decomposition(C):
    dimension = C.shape[0]
    A = np.zeros((dimension, dimension))
    A[0, 0] = np.sqrt(C[0, 0])

    for i in range(1, dimension):   
        A[0, i] = C[0, i]/A[0, 0]

    for i in range(1, dimension-1):
        for j in range(i, dimension):
            sum1 = 0
            sum2 = 0
            for k in range(0, i):
                sum1  += (A[k, i]**2)
                sum2 += (A[k, i]*A[k, j])
                A[i, i] = np.sqrt(C[i, i] - sum1)
                A[i, j] = (C[i, j] - sum2)/A[i, i]

    sum3 = 0
    for k in range(0, dimension):
        sum3 += A[k, dimension-1]**2

    A[dimension-1, dimension-1] = np.sqrt(C[dimension-1, dimension-1] - sum3)
    return A

#Price the rainbow option
def rainbow_option(K, r, T, num_simulation, n, S0, q, sigma, rho):
    C = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            if i == j:
                C[i, i] = sigma[i]*sigma[i]*T
            else:
                C[i, j] = rho[i, j]*sigma[i]*sigma[j]*T

    payoff = np.zeros((num_simulation, 1))
    z = np.zeros((num_simulation, n))
    A = cholesky_decomposition(C)
    for i in range(n):
        z[:, i] = norm.rvs(0, 1, num_simulation)
    r_ = z.dot(A)

    for i in range(n):
        r_[:, i] = r_[:, i]+np.log(S0[i])+(r-q[i]-0.5*sigma[i]*sigma[i])*T
    st = np.exp(r_)
    
    for i in range(num_simulation):
        payoff[i] = max((max(st[i, :])-K), 0)
    
    return np.exp(-r*T)*np.mean(payoff)

#MonteCarlo simulation
def Monte_Carlo_for_rainbow_option(K, r, T, num_simulation, num_repetitions, n, S0, q, sigma, rho):
    call = []
    for i in range(num_repetitions):
        call.append(rainbow_option(K, r, T, num_simulation, n, S0, q, sigma, rho))
    sd = np.std(call)
    mean = np.mean(call)
    lower = mean-2*sd
    upper = mean+2*sd
    return mean, lower, upper

n = 2
rho = np.array([[1, 1],
                [1, 1]])  
print('the value of rainbow option is',Monte_Carlo_for_rainbow_option(K, r, T, num_simulation, num_repetitions, n, S0, q, sigma, rho)[0])
print('the 95% confidence interval is',Monte_Carlo_for_rainbow_option(K, r, T, num_simulation, num_repetitions, n, S0, q, sigma, rho)[1]  
                                      ,Monte_Carlo_for_rainbow_option(K, r, T, num_simulation, num_repetitions, n, S0, q, sigma, rho)[2])
    

the value of rainbow option is 11.915264321074138
the 95% confidence interval is 11.468200644763122 12.418602187065279


In [5]:
n = 2
rho = np.array([[1, -1],
                [-1, 1]])  
print('the value of rainbow option is',Monte_Carlo_for_rainbow_option(K, r, T, num_simulation, num_repetitions, n, S0, q, sigma, rho)[0])
print('the 95% confidence interval is',Monte_Carlo_for_rainbow_option(K, r, T, num_simulation, num_repetitions, n, S0, q, sigma, rho)[1]  
                                      ,Monte_Carlo_for_rainbow_option(K, r, T, num_simulation, num_repetitions, n, S0, q, sigma, rho)[2])

the value of rainbow option is 23.995537010806046
the 95% confidence interval is 23.533462604535515 24.511646071218127


In [7]:
n = 5
rho = np.array([[1, .5, .5, .5, .5],
                [.5, 1, .5, .5, .5],
                [.5, .5, 1, .5, .5],
                [.5, .5, .5, 1, .5],
                [.5, .5, .5, .5, 1]])  
print('the value of rainbow option is',Monte_Carlo_for_rainbow_option(K, r, T, num_simulation, num_repetitions, n, S0, q, sigma, rho)[0])
print('the 95% confidence interval is',Monte_Carlo_for_rainbow_option(K, r, T, num_simulation, num_repetitions, n, S0, q, sigma, rho)[1]  
                                      ,Monte_Carlo_for_rainbow_option(K, r, T, num_simulation, num_repetitions, n, S0, q, sigma, rho)[2])

the value of rainbow option is 30.359861334782543
the 95% confidence interval is 29.625346317285075 31.017914408171414


In [9]:
#Bonus 1
#Antithetic Variate Approach and moment Matching to price rainbow option
n = 5
rho = np.array([[1, .5, .5, .5, .5],
                [.5, 1, .5, .5, .5],
                [.5, .5, 1, .5, .5],
                [.5, .5, .5, 1, .5],
                [.5, .5, .5, .5, 1]])  
                
def rainbow_option_variance_reduction(K, r, T, num_simulation, n, S0, q, sigma, rho):
    C = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            if i == j:
                C[i, i] = sigma[i]*sigma[i]*T
            else:
                C[i, j] = rho[i, j]*sigma[i]*sigma[j]*T

    payoff = np.zeros((num_simulation, 1))
    z = np.zeros((num_simulation, n))
    A = cholesky_decomposition(C)
    for i in range(n):
        z[0:int(num_simulation/2), i] = norm.rvs(0, 1, int(num_simulation/2))   #Antithetic Variate Approach
        z[int(num_simulation/2):, i] = -z[0:int(num_simulation/2), i] 
        z[:, i] = z[:, i]-np.mean(z[:, i])/np.std(z[:, i])                      #Moment Matching
    r_ = z.dot(A)

    for i in range(n):
        r_[:, i] = r_[:, i]+np.log(S0[i])+(r-q[i]-0.5*sigma[i]*sigma[i])*T
    st = np.exp(r_)
    
    for i in range(num_simulation):
        payoff[i] = max((max(st[i, :])-K), 0)
    
    return np.exp(-r*T)*np.mean(payoff)

#MonteCarlo simulation
def Monte_Carlo_for_rainbow_option_variance_reduction(K, r, T, num_simulation, num_repetitions, n, S0, q, sigma, rho):
    call = []
    for i in range(num_repetitions):
        call.append(rainbow_option_variance_reduction(K, r, T, num_simulation, n, S0, q, sigma, rho))
    sd = np.std(call)
    mean = np.mean(call)
    lower = mean-2*sd
    upper = mean+2*sd
    return mean, lower, upper

print('the value of rainbow option is',Monte_Carlo_for_rainbow_option_variance_reduction(K, r, T, num_simulation, num_repetitions, n, S0, q, sigma, rho)[0])
print('the 95% confidence interval is',Monte_Carlo_for_rainbow_option_variance_reduction(K, r, T, num_simulation, num_repetitions, n, S0, q, sigma, rho)[1]  
                                      ,Monte_Carlo_for_rainbow_option_variance_reduction(K, r, T, num_simulation, num_repetitions, n, S0, q, sigma, rho)[2])

the value of rainbow option is 30.406035285444283
the 95% confidence interval is 29.998003286277807 30.657992047627605


In [10]:
#Bonus 2
#Inverse Cholesky Decomposition to price rainbow option
import pandas as pd
from numpy.linalg import inv
n = 5
rho = np.array([[1, .5, .5, .5, .5],
                [.5, 1, .5, .5, .5],
                [.5, .5, 1, .5, .5],
                [.5, .5, .5, 1, .5],
                [.5, .5, .5, .5, 1]])  
                
def rainbow_option_inverse_cholesky_decomposition(K, r, T, num_simulation, n, S0, q, sigma, rho):
    C = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            if i == j:
                C[i, i] = sigma[i]*sigma[i]*T
            else:
                C[i, j] = rho[i, j]*sigma[i]*sigma[j]*T

    payoff = np.zeros((num_simulation, 1))
    z = np.zeros((num_simulation, n))
    for i in range(n):
        z[:, i] = norm.rvs(0, 1, num_simulation)
        z[:, i] = z[:, i]-np.mean(z[:, i])
    COV = pd.DataFrame(z).cov().values
    A_ = cholesky_decomposition(COV) 
    z_ = z.dot(inv(A_))             #Inverse Cholesky Decomposition
    A = cholesky_decomposition(C)
    r_ = z_.dot(A)

    for i in range(n):
        r_[:, i] = r_[:, i]+np.log(S0[i])+(r-q[i]-0.5*sigma[i]*sigma[i])*T
    st = np.exp(r_)
    
    for i in range(num_simulation):
        payoff[i] = max((max(st[i, :])-K), 0)
    
    return np.exp(-r*T)*np.mean(payoff)

#MonteCarlo simulation
def Monte_Carlo_for_rainbow_option_inverse_cholesky(K, r, T, num_simulation, num_repetitions, n, S0, q, sigma, rho):
    call = []
    for i in range(num_repetitions):
        call.append(rainbow_option_inverse_cholesky_decomposition(K, r, T, num_simulation, n, S0, q, sigma, rho))
    sd = np.std(call)
    mean = np.mean(call)
    lower = mean-2*sd
    upper = mean+2*sd
    return mean, lower, upper

print('the value of rainbow option is',Monte_Carlo_for_rainbow_option_inverse_cholesky(K, r, T, num_simulation, num_repetitions, n, S0, q, sigma, rho)[0])
print('the 95% confidence interval is',Monte_Carlo_for_rainbow_option_inverse_cholesky(K, r, T, num_simulation, num_repetitions, n, S0, q, sigma, rho)[1]  
                                      ,Monte_Carlo_for_rainbow_option_inverse_cholesky(K, r, T, num_simulation, num_repetitions, n, S0, q, sigma, rho)[2])


the value of rainbow option is 30.343053078200832
the 95% confidence interval is 30.201765320639755 30.64440756180457
